# Import RPB model along with other utility functions

In [ ]:
from RPB_model import *

# Creating model for a single side/section of the RPB

In [ ]:
# Create model instance. Currently, mode can be either "adsorption" or "desorption" which sets the boundary conditions for each case.
# m=RPB_model(mode="adsorption")
# or
m=RPB_model(mode="adsorption", gas_flow_direction=1)

# Custom initialization routine. Uses a homotopy routine to adjust the initialization factors so that all equations are 'active'.
single_section_init(m)
# homotopy_init_routine(m)

In [ ]:
list(m.z)

In [ ]:
# Plotting the results/profiles

plotting(m)

In [ ]:
# Some various utility functions that I have been using to check model performance
evaluate_MB_error(m)

print(f'CO2 Capture = {m.CO2_capture():.3}')

check_scaling(m)

# degen_hunter(m)

# jac, variables, constraints = scaling_script(m)

In [ ]:
variables[3870].pprint()

In [ ]:
constraints[4076].pprint()

In [ ]:
print(jac.getcol(2921))

# Creating a full RPB model

In [ ]:
RPB = full_model_creation(lean_temp_connection=True, configuration = "counter-current")
init_routine_1(RPB)

In [ ]:
fix_capture_and_solve(RPB, capture=0.95)

In [ ]:
report(RPB)

In [ ]:
evaluate_MB_error(RPB.ads)

In [ ]:
check_scaling(RPB)

In [ ]:
jac, variables, constraints = scaling_script(RPB)

# Save and load model

In [ ]:
# save model
to_json(RPB, fname="counter current base case 090823.json.gz", gz=True, human_read=False)

In [ ]:
# create model, load, and solve
RPB = full_model_creation(lean_temp_connection=True, configuration = "counter-current")

from_json(RPB, fname="counter current base case 090823.json.gz", gz=True)

solver = SolverFactory("ipopt")
solver.options = {
    "max_iter": 1000,
    "bound_push": 1e-22,
    # "halt_on_ampl_error": "yes",
}
solver.solve(RPB, tee=True).write()

Optimization Testing

In [ ]:
# create regularization parameter for the objective function
RPB.alpha_obj = Param(initialize=0.9, mutable=True)

# add objective
@RPB.Expression()
def obj(RPB):
    return RPB.alpha_obj * RPB.energy_requirement/10 - (1 - RPB.alpha_obj) * RPB.productivity/20

RPB.objective = Objective(expr=RPB.obj)

In [ ]:
RPB.ads.L.setlb(0.5)
RPB.ads.L.setub(10)
RPB.ads.L.pprint()

In [ ]:
RPB.ads.Tx.setlb(25+273)
RPB.ads.Tx.setub(95+273)
RPB.ads.Tx.pprint()

In [ ]:
RPB.des.Tx.setlb(100+273)
RPB.des.Tx.setub(160+273)
RPB.des.Tx.pprint()

In [ ]:
RPB.des.P_in.setub(1.15)
RPB.des.P_in.setlb(1.02)
RPB.des.P_in.pprint()

In [ ]:
degrees_of_freedom(RPB)

In [ ]:
RPB.ads.w_rpm.setlb(0.5)
RPB.ads.w_rpm.setub(5)
RPB.ads.w_rpm.pprint()

In [ ]:
# free up decision variables
# RPB.ads.L.unfix()
RPB.ads.theta.unfix()
# RPB.des.P_in.unfix()
# RPB.ads.Tx.unfix()
# RPB.des.Tx.unfix()

In [ ]:
degrees_of_freedom(RPB)

In [ ]:
# solve
solver = SolverFactory("ipopt")
solver.options = {
    "max_iter": 1000,
    "bound_push": 1e-8,
    # "halt_on_ampl_error": "yes",
    "tol": 1e-4,
    "max_cpu_time": 5*60,
    "mu_strategy": "adaptive",
}
solver.solve(RPB, tee=True).write()

In [ ]:
report(RPB)

In [ ]:
[RPB.ads.qCO2_eq[z,o]() for z,o in RPB.ads.qCO2_eq.keys()]

In [ ]:
RPB.ads.Tx()

In [ ]:
full_contactor_plotting(RPB)

In [ ]:
RPB.ads.Q_ghx_tot_kW()

In [ ]:
list(RPB.ads.z)